In [1]:
import pandas as pd
from fpdf import FPDF as pdf
import datetime
from typing import Tuple, Dict
import numpy as np

In [2]:
from analytics_messenger.email_messenger import send_email
from analytics_messenger.database.data_read import DataRead

from analytics_messenger.queries.queries.query_lojas import QUERY_LOJAS
from analytics_messenger.queries.queries.query_metas import QUERY_METAS
from analytics_messenger.queries.queries.query_parcelas import QUERY_PARCELAS
from analytics_messenger.queries.queries.query_vendas import QUERY_VENDAS
from analytics_messenger.queries.queries.query_vendedores import QUERY_VENDEDORES

# Funções

## Recebe o Dataframe e o adiciona em uma página de um pdf

In [3]:
from fpdf import XPos, YPos
from numpy import recarray


# Get Width of Columns
def get_col_widths(cell_width: str, pdf: pdf, table_data, data):
    if cell_width == "even":
        col_width = (
            pdf.epw / len(data[0]) - 1
        )  # distribute content evenly   # epw = effective page width (width of page not including margins)
    elif cell_width == "uneven":
        col_widths = []

        # searching through columns for largest sized cell (not rows but cols)
        for col in range(len(table_data[0])):  # for every row
            longest = 0
            for row in range(len(table_data)):
                cell_value = str(table_data[row][col])
                value_length = pdf.get_string_width(cell_value)
                if value_length > longest:
                    longest = value_length
            col_widths.append(longest + 4)  # add 4 for padding
        col_width = col_widths

        ### compare columns

    elif isinstance(cell_width, list):
        col_width = cell_width  # TODO: convert all items in list to int
    else:
        # TODO: Add try catch
        col_width = int(cell_width)
    return col_width


def create_table(
    pdf: pdf,
    table_data: recarray,
    title="",
    data_size=10,
    title_size=12,
    align_data="L",
    align_header="L",
    cell_width="even",
    x_start="x_default",
    emphasize_data=[],
    emphasize_style=None,
    emphasize_color=(0, 0, 0),
):
    """
    table_data:
                list of lists with first element being list of headers
    title:
                (Optional) title of table (optional)
    data_size:
                the font size of table data
    title_size:
                the font size fo the title of the table
    align_data:
                align table data
                L = left align
                C = center align
                R = right align
    align_header:
                align table data
                L = left align
                C = center align
                R = right align
    cell_width:
                even: evenly distribute cell/column width
                uneven: base cell size on lenght of cell/column items
                int: int value for width of each cell/column
                list of ints: list equal to number of columns with the widht of each cell / column
    x_start:
                where the left edge of table should start
    emphasize_data:
                which data elements are to be emphasized - pass as list
                emphasize_style: the font style you want emphaized data to take
                emphasize_color: emphasize color (if other than black)

    """
    default_style = pdf.font_style
    if emphasize_style == None:
        emphasize_style = default_style
    # default_font = pdf.font_family
    # default_size = pdf_file_file.font_size_pt
    # default_style = pdf.font_style
    # default_color = pdf.color # This does not work

    # Convert dict to lol
    # Why? because i built it with lol first and added dict func after
    # Is there performance differences?
    if isinstance(table_data, dict):
        header = [key for key in table_data]
        data = []
        for key in table_data:
            value = table_data[key]
            data.append(value)
        # need to zip so data is in correct format (first, second, third --> not first, first, first)
        data = [list(a) for a in zip(*data)]

    else:
        header = table_data[0]
        data = table_data[1:]

    line_height = pdf.font_size * 2.5

    col_width = get_col_widths(cell_width=cell_width, pdf=pdf, table_data=table_data, data=data)

    pdf.set_font(size=title_size)

    # Get starting position of x
    # Determin width of table to get x starting point for centred table
    if x_start == "C":
        table_width = 0
        if isinstance(col_width, list):
            for width in col_width:
                table_width += width
        else:  # need to multiply cell width by number of cells to get table width
            table_width = col_width * len(table_data[0])
        # Get x start by subtracting table width from pdf width and divide by 2 (margins)
        margin_width = pdf.w - table_width
        # TODO: Check if table_width is larger than pdf width

        center_table = margin_width / 2  # only want width of left margin not both
        x_start = center_table
        pdf.set_x(x_start)
    elif isinstance(x_start, int):
        pdf.set_x(x_start)
    elif x_start == "x_default":
        x_start = pdf.set_x(pdf.l_margin)

    # TABLE CREATION #

    # add title
    if title != "":
        pdf.multi_cell(
            0, line_height, title, border=0, align="j", new_x=XPos.RIGHT, new_y=YPos.TOP, max_line_height=pdf.font_size
        )
        pdf.ln(line_height)  # move cursor back to the left margin

    pdf.set_font(size=data_size)
    # add header
    y1 = pdf.get_y()
    if x_start:
        x_left = x_start
    else:
        x_left = pdf.get_x()
    x_right = pdf.epw + x_left
    if not isinstance(col_width, list):
        if x_start:
            pdf.set_x(x_start)
        for datum in header:
            pdf.multi_cell(
                col_width,
                line_height,
                datum,
                border=0,
                align=align_header,
                new_x=XPos.RIGHT,
                new_y=YPos.TOP,
                max_line_height=pdf.font_size,
            )
            x_right = pdf.get_x()
        pdf.ln(line_height)  # move cursor back to the left margin
        y2 = pdf.get_y()
        pdf.line(x_left, y1, x_right, y1)
        pdf.line(x_left, y2, x_right, y2)

        for row in data:
            if x_start:  # not sure if I need this
                pdf.set_x(x_start)
            for datum in row:
                if datum in emphasize_data:
                    pdf.set_text_color(*emphasize_color)
                    pdf.set_font(style=emphasize_style)
                    pdf.multi_cell(
                        col_width,
                        line_height,
                        datum,
                        border=0,
                        align=align_data,
                        new_x=XPos.RIGHT,
                        new_y=YPos.TOP,
                        max_line_height=pdf.font_size,
                    )
                    pdf.set_text_color(0, 0, 0)
                    pdf.set_font(style=default_style)
                else:
                    pdf.multi_cell(
                        col_width,
                        line_height,
                        datum,
                        border=0,
                        align=align_data,
                        new_x=XPos.RIGHT,
                        new_y=YPos.TOP,
                        max_line_height=pdf.font_size,
                    )  # ln = 3 - move cursor to right with same vertical offset # this uses an object named pdf
            pdf.ln(line_height)  # move cursor back to the left margin

    else:
        if x_start:
            pdf.set_x(x_start)
        for i in range(len(header)):
            datum = header[i]
            pdf.multi_cell(
                col_width[i],
                line_height,
                datum,
                border=0,
                align=align_header,
                new_x=XPos.RIGHT,
                new_y=YPos.TOP,
                max_line_height=pdf.font_size,
            )
            x_right = pdf.get_x()
        pdf.ln(line_height)  # move cursor back to the left margin
        y2 = pdf.get_y()
        pdf.line(x_left, y1, x_right, y1)
        pdf.line(x_left, y2, x_right, y2)

        for i in range(len(data)):
            if x_start:
                pdf.set_x(x_start)
            row = data[i]
            for i in range(len(row)):
                datum = row[i]
                if not isinstance(datum, str):
                    datum = str(datum)
                adjusted_col_width = col_width[i]
                if datum in emphasize_data:
                    pdf.set_text_color(*emphasize_color)
                    pdf.set_font(style=emphasize_style)
                    pdf.multi_cell(
                        adjusted_col_width,
                        line_height,
                        datum,
                        border=0,
                        align=align_data,
                        new_x=XPos.RIGHT,
                        new_y=YPos.TOP,
                        max_line_height=pdf.font_size,
                    )
                    pdf.set_text_color(0, 0, 0)
                    pdf.set_font(style=default_style)
                else:
                    pdf.multi_cell(
                        adjusted_col_width,
                        line_height,
                        datum,
                        border=0,
                        align=align_data,
                        new_x=XPos.RIGHT,
                        new_y=YPos.TOP,
                        max_line_height=pdf.font_size,
                    )  # ln = 3 - move cursor to right with same vertical offset # this uses an object named pdf
            pdf.ln(line_height)  # move cursor back to the left margin
    y3 = pdf.get_y()
    pdf.line(x_left, y3, x_right, y3)

## Cria o arquivo PDF

In [ ]:
# import dataclasses
# from typing import Callable, List, Optional, Union

# @dataclass
# class IPageContent:
#     title: str
#     content: List[pd.DataFrame]

# @dataclass
# class IReportContent:
#     nome_loja: str
#     report_date: datetime.datetime
#     content: Dict[str,List[pd.DataFrame]]

# @dataclass
# class IReportStoreContent:
#     nome_loja: str
#     report_date: datetime.datetime
#     content: IPageContent
#     tags: Optional[List[str]] = None
#     responses: Optional[list] = None

In [ ]:
# {
#     'Report Diário':[df_store_formated],
#     'Report Vendedores':[df_vendedor_formated_month,df_vendedor_formated_day]
# }


# 'Relatorio  completo': {
#     '1':{'title':'vendas por loja', 'content': [df_store_formated]},
#     '2':{'title':'por vendedor', 'content': [df_vendedor_formated_month,df_vendedor_formated_day]}
# }

In [4]:
from typing import List, Dict


def create_pdf_report(
    filename: str,
    nome_loja: str,
    yesterday_date: datetime.datetime,
    # dataframes_list: Dict[str, List[pd.DataFrame]],
    df_store_formated: pd.DataFrame,
    df_vendedor_formated_month: pd.DataFrame,
    df_vendedor_formated_day: pd.DataFrame,
):

    data_1 = df_store_formated.astype(str).to_records()
    data_2 = df_vendedor_formated_month.astype(str).to_records()
    data_3 = df_vendedor_formated_day.astype(str).to_records()

    pdf_file = pdf()
    pdf_file.set_font("Times", size=9)

    pdf_file.add_page(orientation="l")
    create_table(
        pdf=pdf_file,
        table_data=data_1,
        title=f'KPIs Loja {nome_loja} - {yesterday_date.strftime("%d.%m.%Y")}',
        cell_width="even",
    )

    pdf_file.add_page(orientation="l")
    create_table(pdf=pdf_file, table_data=data_2, title="KPI Vendedor Mês", cell_width=23)
    pdf_file.ln()
    create_table(
        pdf=pdf_file, table_data=data_3, title=f'KPIs Vendedor Dia {yesterday_date.strftime("%d.%m.%Y")}', cell_width=23
    )

    pdf_file.output(f"{filename}.pdf")

In [ ]:
from abc import ABCMeta, abstractmethod


class ReportGenerator(metaclass=ABCMeta):
    @abstractmethod
    def __init__(self) -> None:
        pass

    @abstractmethod
    def generate_report(self, Dict) -> None:
        pass

In [ ]:
class PdfGenerator(ReportGenerator):
    def __init__():
        pass

## Lê e armazena os dados do BQ

In [5]:
class KpiDataManagerA:
    def __init__(self, delta_days: int = 1) -> None:
        self.delta_days = delta_days
        self.yesterday_date = datetime.datetime.today() - datetime.timedelta(days=self.delta_days)
        self.yesterday_date_str = self.yesterday_date.strftime("%Y-%m-%d")

    @property
    def all_dataframes(self) -> Dict[str, pd.DataFrame]:
        return {
            "df_metas": self.df_metas,
            "df_lojas": self.df_lojas,
            "df_vendedores": self.df_vendedores,
            "df_vendas": self.df_vendas,
            "df_parcelas": self.df_parcelas,
            "df_vendas_pdv": self.df_vendas_pdv,
            "df_parcelas_with_displaycode": self.df_parcelas_with_displaycode,
            "df_nome_vendedor": self.df_nome_vendedor,
        }

    def fetch_and_build_datasets(self) -> bool:
        print("self._fetch_data_from_bigquery()")
        self._fetch_data_from_bigquery()

        print("self._build_df_vendas_pdv()")
        self._build_df_vendas_pdv()

        print("self._build_df_parcelas_with_displaycode()")
        self._build_df_parcelas_with_displaycode()

        print("self.df_nome_vendedor()")
        self.df_nome_vendedor = (
            self.df_vendas_pdv.groupby(["vendedor", "cpf_vendedor_inteiro"])["is_frete"]
            .max()
            .reset_index()
            .drop(columns="is_frete")
        )

        print("self.df_kpis_loja")
        kpis_loja_dict = {}
        for kpi_period in ["Dia", "Mês", "Ano"]:
            kpis_loja_dict[f"kpis_loja_{kpi_period.lower()}"] = self._build_kpis_dataframe(
                kpi_type="loja", kpi_period=kpi_period
            )
        self.df_kpis_loja = pd.concat(list(kpis_loja_dict.values()))

        print("self.df_kpis_vendedor")
        kpis_vendedor_dict = {}
        for kpi_period in ["Dia", "Mês", "Ano"]:
            kpis_vendedor_dict[f"kpis_vendedor_{kpi_period.lower()}"] = self._build_kpis_dataframe(
                kpi_type="vendedor", kpi_period=kpi_period
            )

        df_kpis_vendedor = pd.concat(list(kpis_vendedor_dict.values()))
        df_kpis_vendedor = df_kpis_vendedor.dropna(subset="cpf_vendedor_inteiro")
        mask = df_kpis_vendedor["cpf_vendedor_inteiro"] != "Devolução"
        self.df_kpis_vendedor = df_kpis_vendedor[mask]

        return True

    def _fetch_data_from_bigquery(self) -> None:
        print("QUERY_METAS")
        self.df_metas = DataRead.from_bigquery(QUERY_METAS).rename(
            columns={"cpf_vendedor": "cpf_vendedor_inteiro", "loja": "distributorId"}
        )

        print("QUERY_LOJAS")
        self.df_lojas = DataRead.from_bigquery(QUERY_LOJAS)

        print("QUERY_VENDEDORES")
        self.df_vendedores = DataRead.from_bigquery(QUERY_VENDEDORES)

        print("QUERY_VENDAS")
        self.df_vendas = DataRead.from_bigquery(QUERY_VENDAS)

        print("QUERY_PARCELAS")
        self.df_parcelas = DataRead.from_bigquery(QUERY_PARCELAS)

    def _build_df_vendas_pdv(self) -> None:
        df_vendas_temp = self.df_vendas.copy()
        PDV_TYPE_LIST = ["STORE", "PI - 360", "PI", "EP", "Troca", "Devolução"]
        df_vendas_temp["is_frete"] = df_vendas_temp["tipo_transacao"] == "frete"
        df_vendas_temp = df_vendas_temp.sort_values(["displaycode", "is_frete"]).reset_index(drop=True)
        df_vendas_temp["tipo_transacao"] = df_vendas_temp["tipo_transacao"].replace("frete", None)
        df_vendas_temp["tipo_transacao"] = df_vendas_temp["tipo_transacao"].fillna(
            df_vendas_temp["tipo_transacao"].shift(1)
        )
        df_vendas_temp["venda_cupom"] = ~df_vendas_temp["cupom_vendedor"].isnull()
        mask_pdv = df_vendas_temp["tipo_transacao"].isin(PDV_TYPE_LIST)
        df_vendas_pdv = df_vendas_temp[mask_pdv]
        df_vendas_pdv.loc[:, "tipo_transacao"] = df_vendas_pdv["tipo_transacao"].replace("PI - 360", "STORE")
        df_vendas_pdv.loc[:, "tipo_transacao"] = df_vendas_pdv["tipo_transacao"].replace("PI", "STORE")
        df_vendas_pdv.loc[:, "tipo_transacao"] = df_vendas_pdv["tipo_transacao"].replace("EP", "STORE")

        # df_venda_cupom = df_vendas[df_vendas['venda_cupom']]
        # df_venda_cupom.loc[:, 'tipo_transacao'] = 'CUPOM'
        # df_vendas_pdv = pd.concat([df_vendas_pdv, df_venda_cupom])
        self.df_vendas_pdv = df_vendas_pdv.reset_index(drop=True)
        return None

    def _build_df_parcelas_with_displaycode(self):
        df_vendedor_displaycode = (
            self.df_vendas_pdv.groupby(["displaycode", "cpf_vendedor_inteiro", "distributorId"])["year"]
            .count()
            .reset_index()
            .drop(columns="year")
        )
        df_parcelas_with_displaycode = self.df_parcelas.merge(df_vendedor_displaycode, how="left", on="displaycode")
        self.df_parcelas_with_displaycode = df_parcelas_with_displaycode
        return None

    def _build_kpis_dataframe(self, kpi_period: str, kpi_type: str, **kwargs) -> pd.DataFrame:

        mask_tickets, mask_vendas, mask_parcelas, mask_meta = self._build_masks(
            kpi_period=kpi_period,
            kpi_type=kpi_type,
        )

        df_kpis = self._merge_dataframes(
            kpi_period=kpi_period,
            kpi_type=kpi_type,
            mask_vendas=mask_vendas,
            mask_tickets=mask_tickets,
            mask_parcelas=mask_parcelas,
            mask_meta=mask_meta,
        )

        if kpi_type == "loja":
            if kpi_period == "Ano":
                df_kpis["tkts_per_day"] = df_kpis["tkts"] / self.yesterday_date.timetuple().tm_yday
            if kpi_period == "Mês":
                df_kpis["tkts_per_day"] = df_kpis["tkts"] / self.yesterday_date.day
            if kpi_period == "Dia":
                df_kpis["tkts_per_day"] = df_kpis["tkts"]

        elif kpi_type == "vendedor":
            if kpi_period == "Mês":
                df_kpis["tkts_per_day"] = df_kpis["tkts"] / df_kpis["days_of_work"].round(2)
            else:
                df_kpis["tkts_per_day"] = df_kpis["tkts"]

        df_kpis["type"] = kpi_period

        df_kpis["mkp"] = (df_kpis["net_value"] / df_kpis["store_cost"]).round(2)
        df_kpis["tkt_medio"] = df_kpis["net_value"] / df_kpis["tkts"].round(2)
        df_kpis["meta_percentual"] = (df_kpis["net_value"] / df_kpis["meta"] * 100).round(2)
        df_kpis["PA"] = df_kpis["amount"] / df_kpis["tkts"].round(2)
        df_kpis["discount_percentual"] = ((df_kpis["discount_value"] / df_kpis["net_value"]) * 100).round(2)

        return df_kpis

    def _build_masks(
        self,
        kpi_period: str,
        kpi_type: str,
    ) -> Tuple[pd.Series, pd.Series, pd.Series, pd.Series]:

        mask_tickets = (
            self.df_vendas_pdv["tipo_transacao"].isin(["STORE", "CUPOM"]) & self.df_vendas_pdv["is_frete"] == False
        )

        mask_vendas = pd.Series(np.ones(self.df_vendas_pdv.shape[0]).astype(bool))

        mask_parcelas = pd.Series(np.ones(self.df_parcelas_with_displaycode.shape[0]).astype(bool))

        if kpi_type == "loja":
            mask_meta = self.df_metas["tipo_meta"] == "Meta Loja PDV"
        elif "vendedor":
            mask_meta = self.df_metas["tipo_meta"] == "Meta Vendedor PDV"

        if kpi_period == "Dia":
            mask_yesterday_vendas = self.df_vendas_pdv["invoice_date"].astype(str) == self.yesterday_date_str
            mask_vendas = mask_yesterday_vendas
            mask_tickets = mask_tickets & mask_yesterday_vendas

            mask_metas_yesterday = self.df_metas["date"].astype(str) == self.yesterday_date_str
            mask_meta = mask_meta & mask_metas_yesterday

            mask_parcelas = self.df_parcelas_with_displaycode["data_emissao_NF"].astype(str) == self.yesterday_date_str
        return mask_tickets, mask_vendas, mask_parcelas, mask_meta

    def _merge_dataframes(
        self,
        kpi_period: str,
        kpi_type: str,
        mask_vendas: pd.Series,
        mask_tickets: pd.Series,
        mask_parcelas: pd.Series,
        mask_meta: pd.Series,
    ) -> pd.DataFrame:

        AGG_DICT = {"discount_value": "sum", "net_value": "sum", "store_cost": "sum"}

        GROUP_BY_COLUMNS_DICT = {
            "Ano": ["distributorId", "year"],
            "Mês": ["distributorId", "year", "month"],
            "Dia": ["distributorId", "year", "month"],
        }

        group_by_columns = GROUP_BY_COLUMNS_DICT[kpi_period]
        if kpi_type == "vendedor":
            group_by_columns = ["cpf_vendedor_inteiro"] + group_by_columns

        df_kpis = self.df_vendas_pdv[mask_vendas].groupby(group_by_columns).agg(AGG_DICT).reset_index()

        df_kpis = df_kpis.merge(
            self.df_parcelas_with_displaycode[mask_parcelas]
            .groupby(group_by_columns)["n_parcelas"]
            .mean()
            .reset_index(),
            on=group_by_columns,
            how="left",
        )

        df_kpis = df_kpis.merge(
            self.df_vendas_pdv[mask_tickets]
            .groupby(group_by_columns)["parent_orderId"]
            .nunique()
            .reset_index()
            .rename(columns={"parent_orderId": "tkts"}),
            how="left",
        )

        df_kpis = df_kpis.merge(
            self.df_vendas_pdv[mask_tickets].groupby(group_by_columns)["amount"].sum().reset_index(), how="left"
        )

        df_kpis = df_kpis.merge(
            self.df_metas[mask_meta].groupby(group_by_columns)["meta"].sum().reset_index(), how="left"
        )

        if kpi_type == "vendedor":
            df_kpis = df_kpis.merge(
                self.df_vendas_pdv.groupby(group_by_columns)["invoice_date"]
                .nunique()
                .reset_index()
                .rename(columns={"invoice_date": "days_of_work"}),
                how="left",
            )

        if kpi_type == "day":
            yesterday_date = datetime.datetime.today() - datetime.timedelta(days=1)
            # df_kpis = df_kpis['day'] = yesterday_date.day
            df_kpis["day"] = yesterday_date.day

        return df_kpis

# Formatando os Dados

In [6]:
class KpiDataFormater:
    def __init__(self) -> None:
        pass

    @classmethod
    def format_all_tables(
        self, id_loja: str, df_kpis_loja: pd.DataFrame, df_kpis_vendedor: pd.DataFrame, df_nome_vendedor: pd.DataFrame
    ):

        df_store_formated = self._format_store_kpis(df_kpis_loja=df_kpis_loja, id_loja=id_loja)

        if df_store_formated.size == 0:
            return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

        df_vendedor_formated_month = self._format_vendedor_kpis_mes(
            df_kpis_vendedor=df_kpis_vendedor, df_nome_vendedor=df_nome_vendedor, id_loja=id_loja
        )

        df_vendedor_formated_day = self._format_vendedor_kpis_dia(
            df_kpis_vendedor=df_kpis_vendedor, df_nome_vendedor=df_nome_vendedor, id_loja=id_loja
        )

        return df_store_formated, df_vendedor_formated_month, df_vendedor_formated_day

    @staticmethod
    def _format_store_kpis(df_kpis_loja: pd.DataFrame, id_loja: str) -> pd.DataFrame:

        mask = df_kpis_loja["distributorId"] == id_loja

        df_store = df_kpis_loja[mask].fillna(0).copy()
        df_store_current_month = df_store.loc[df_store.groupby(["type"])["month"].idxmax()].drop_duplicates()

        rename_dict = {
            "net_value": "Venda Loja (PDV)",
            "meta": "Meta Loja (PDV)",
            "meta_percentual": "Meta % (PDV)",
            "mkp": "Markup",
            "tkts_per_day": "Tickets/Dia",
            "PA": "Peças por Atendimento",
            "tkt_medio": "Ticket Médio",
            # 'n_parcelas':'Parcela Média',
            "discount_percentual": "Desconto Médio %",
            "type": "Período",
        }

        format_money = "R$ {:,.0f}"
        format_percentage = "{:,.0f}%"

        format_dict = {
            "Venda Loja (PDV)": format_money,
            "Meta Loja (PDV)": format_money,
            "Meta % (PDV)": format_percentage,
            "Markup": "{:,.2f}",
            "Tickets/Dia": "{:,.1f}",
            "Peças por Atendimento": "{:,.1f}",
            "Ticket Médio": format_money,
            # 'Parcela Média':'{:,.1f}',
            "Desconto Médio %": format_percentage,
        }

        df_store_formated = df_store_current_month[rename_dict.keys()].round(2).rename(columns=rename_dict)

        for key, format in format_dict.items():
            df_store_formated[key] = df_store_formated[key].map(format.format)

            if format == format_money:
                df_store_formated[key] = df_store_formated[key].str.replace(",", ".")

        try:
            df_store_formated = df_store_formated.set_index("Período").reset_index().T[[1, 2, 0]]
        except KeyError:
            df_store_formated = pd.DataFrame()
        return df_store_formated

    @staticmethod
    def _format_vendedor_kpis_mes(
        df_kpis_vendedor: pd.DataFrame, df_nome_vendedor: pd.DataFrame, id_loja: str
    ) -> pd.DataFrame:

        mask = df_kpis_vendedor["distributorId"] == id_loja
        df_vendedor = df_kpis_vendedor[mask].reset_index(drop=True)
        df_vendedor["month"] = df_vendedor["month"].fillna(0)

        mask_ano_dia = df_vendedor["type"].isin(["Mês"])
        mask_mes = df_vendedor["month"] == df_vendedor["month"].max()
        mask = mask_ano_dia & mask_mes
        df_vendedor_mes: pd.DataFrame = df_vendedor[mask].reset_index(drop=True)
        df_vendedor_mes["normalized_net_value"] = (
            df_vendedor_mes["net_value"] * 100 / df_vendedor_mes["net_value"].max().round(2)
        )
        df_vendedor_mes["net_value_share"] = (
            df_vendedor_mes["net_value"] * 100 / df_vendedor_mes["net_value"].max().round(2)
        )
        df_vendedor_mes = df_nome_vendedor.merge(df_vendedor_mes, on="cpf_vendedor_inteiro").drop(
            columns="cpf_vendedor_inteiro"
        )

        rename_dict = {
            "vendedor": "Vendedor",
            "days_of_work": "Dias Trabalhados",
            "net_value": "Acumulado no Mês",
            "normalized_net_value": "Homogeneidade",
            "meta": "Meta do Mês",
            "meta_percentual": "Meta %",
            "mkp": "Markup",
            "tkts_per_day": "Tickets/Dia",
            "PA": "Peças por Atendimento",
            "tkt_medio": "Ticket Médio",
            # 'n_parcelas':'Parcela Média',
            "discount_percentual": "Desconto Médio %",
            # 'type':'Período'
        }

        format_money = "R$ {:,.0f}"
        format_percentage = "{:,.0f}%"

        format_dict = {
            "Acumulado no Mês": format_money,
            "Homogeneidade": format_percentage,
            "Meta do Mês": format_money,
            "Meta %": format_percentage,
            "Markup": "{:,.2f}",
            "Tickets/Dia": "{:,.1f}",
            "Peças por Atendimento": "{:,.1f}",
            "Ticket Médio": format_money,
            # 'Parcela Média':'{:,.1f}',
            "Desconto Médio %": format_percentage,
        }

        df_vendedor_mes_formatado = (
            df_vendedor_mes[rename_dict.keys()]
            .sort_values("normalized_net_value", ascending=False)
            .round(2)
            .rename(columns=rename_dict)
        )

        for key, format in format_dict.items():
            df_vendedor_mes_formatado[key] = df_vendedor_mes_formatado[key].map(format.format)
            if format == format_money:
                df_vendedor_mes_formatado[key] = df_vendedor_mes_formatado[key].str.replace(",", ".")

        df_vendedor_mes_formatado = df_vendedor_mes_formatado.reset_index(drop=True).T.reset_index().T

        return df_vendedor_mes_formatado.set_index(0)

    @staticmethod
    def _format_vendedor_kpis_dia(
        df_kpis_vendedor: pd.DataFrame, df_nome_vendedor: pd.DataFrame, id_loja: str
    ) -> pd.DataFrame:

        mask = df_kpis_vendedor["distributorId"] == id_loja
        df_vendedor = df_kpis_vendedor[mask].reset_index(drop=True)
        df_vendedor["month"] = df_vendedor["month"].fillna(0)

        mask_dia = df_vendedor["type"].isin(["Dia"])
        df_vendedor_dia = df_vendedor[mask_dia].reset_index(drop=True)
        df_vendedor_dia["normalized_net_value"] = (
            df_vendedor_dia["net_value"] * 100 / df_vendedor_dia["net_value"].max().round(2)
        )
        df_vendedor_dia["net_value_share"] = (
            df_vendedor_dia["net_value"] * 100 / df_vendedor_dia["net_value"].max().round(2)
        )
        df_vendedor_dia = df_nome_vendedor.merge(df_vendedor_dia, on="cpf_vendedor_inteiro").drop(
            columns="cpf_vendedor_inteiro"
        )

        rename_dict = {
            "vendedor": "Vendedor",
            "net_value": "Acumulado no Dia",
            "normalized_net_value": "Homogeneidade",
            "meta": "Meta do Dia",
            "meta_percentual": "Meta %",
            "mkp": "Markup",
            "tkts_per_day": "Tickets",
            "PA": "Peças por Atendimento",
            "tkt_medio": "Ticket Médio",
            # 'n_parcelas':'Parcela Média',
            "discount_percentual": "Desconto Médio %",
        }

        format_money = "R$ {:,.0f}"
        format_percentage = "{:,.0f}%"

        format_dict = {
            "Acumulado no Dia": format_money,
            "Homogeneidade": format_percentage,
            "Meta do Dia": format_money,
            "Meta %": format_percentage,
            "Markup": "{:,.2f}",
            "Tickets": "{:,.0f}",
            "Peças por Atendimento": "{:,.1f}",
            "Ticket Médio": format_money,
            # 'Parcela Média':'{:,.1f}',
            "Desconto Médio %": format_percentage,
        }

        df_vendedor_dia_formatado = (
            df_vendedor_dia[rename_dict.keys()]
            .sort_values("normalized_net_value", ascending=False)
            .rename(columns=rename_dict)
        )

        for key, format in format_dict.items():
            df_vendedor_dia_formatado[key] = df_vendedor_dia_formatado[key].map(format.format)
            if format == format_money:
                df_vendedor_dia_formatado[key] = df_vendedor_dia_formatado[key].str.replace(",", ".")

        df_vendedor_dia_formatado = df_vendedor_dia_formatado.reset_index(drop=True).T.reset_index().T
        return df_vendedor_dia_formatado.set_index(0)

In [2]:
from analytics_messenger.data.data_manager import KpiDataManager


kpi_data_manager = KpiDataManager()

kpi_data_manager.fetch_and_build_datasets("local")

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/gcloud-api.json'

In [8]:
kpi_data_manager.df_lojas.to

Index(['loja_group_code', 'loja_id', 'data_criacao', 'data_inauguracao',
       'data_atualizacao', 'safra', 'sss', 'status_comercial', 'tipo', 'canal',
       'marca', 'ativa', 'status', 'nome', 'nome_completo', 'cep', 'estado',
       'metragem', 'empreendimento', 'localizado_shopping', 'regiao', 'cidade',
       'endereco', 'distrito', 'layout', 'regional', 'franqueado',
       'franqueado_email', 'franqueado_telefone', 'status_financeiro', 'email',
       'email_gerente', 'filial_datasul'],
      dtype='object')

In [6]:
kpi_data_manager.all_dataframes.keys()

dict_keys(['df_metas', 'df_lojas', 'df_vendedores', 'df_vendas', 'df_parcelas', 'df_vendas_pdv', 'df_parcelas_with_displaycode', 'df_nome_vendedor'])

In [7]:
import pickle

pickle.dump(kpi_data_manager, open("kpi_data_manager.pkl", "wb"))

In [16]:
import numpy as np

a = np.nan
b = a
a == a

False

In [24]:
a is b

True

In [17]:
np.isnan(a)

np.True_

In [23]:
if np.False_:
    print("oi")

In [12]:
with open("kpi_data_manager.pkl", "rb") as file:
    objeto_lido: KpiDataManager = pickle.load(file)

In [13]:
objeto_lido.all_dataframes.keys()

dict_keys(['df_metas', 'df_lojas', 'df_vendedores', 'df_vendas', 'df_parcelas', 'df_vendas_pdv', 'df_parcelas_with_displaycode', 'df_nome_vendedor'])

In [ ]:
import json

json.loads("test").encode("utf-8")

In [9]:
columns = ["loja_id", "nome", "safra", "data_inauguracao", "tipo", "marca", "metragem", "regional"]
mask_ativas = kpi_data_manager.df_lojas["ativa"] > 0
kpi_data_manager.df_lojas[columns][mask_ativas]

,loja_id,nome,safra,data_inauguracao,tipo,marca,metragem,regional
0,0773,Iguatemi Porto Alegre,2023,2023-10-04,Franquia,mmartan,51.65,Ivan
1,0626,Jaú Shopping,2017,2017-12-14,Franquia,mmartan,46.82,Ivan
2,0741,Partage Shopping Marabá,2020,2020-09-01,Franquia,mmartan,56.58,Ivan
3,0736,Imperial Shopping,2017,2017-12-14,Franquia,mmartan,48.85,Ivan
4,0599,Mooca Plaza Shopping,2017,2017-12-14,Franquia,mmartan,NaN,Ivan
...,...,...,...,...,...,...,...,...
449,2579,Amapá Garden Shopping,2022,2022-12-09,Franquia,artex,75.00,Ivan
450,2575,Shopping São Luis,2022,2022-04-13,Franquia,artex,89.85,Ivan
451,2561,Shopping da Ilha,2021,2021-07-14,Franquia,artex,56.79,Ivan
452,2583,Santa Inês,2023,2023-05-03,Franquia,artex,69.80,Ivan


In [10]:
kpi_data_formater = KpiDataFormater()

df_store_formated, df_vendedor_formated_month, df_vendedor_formated_day = kpi_data_formater.format_all_tables(
    id_loja="1077",
    df_kpis_loja=kpi_data_manager.df_kpis_loja,
    df_kpis_vendedor=kpi_data_manager.df_kpis_vendedor,
    df_nome_vendedor=kpi_data_manager.df_nome_vendedor,
)

In [11]:
df_vendedor_formated_month

""


In [4]:
email_luciano = "luciano.millamonte@coteminas.com.br"
email_clarissa = "clarissa.ferreira@ammovarejo.com.br"
email_claudio = "claudio.oliveira@ammovarejo.com.br"
email_ivan = "ivan.teixeira@ammovarejo.com.br"
email_zeuxis = "zeuxis.guimaraes@coteminas.com.br"
email_rodrigo = "rodrigo.graci@coteminas.com.br"
email_sandra = "sandra.pereira@ammovarejo.com.br"
email_teste = "joao.garcia@ammovarejo.com.br"

In [13]:
regional_loja = {
    "2560": [email_zeuxis],
    "2510": [email_zeuxis],
    "2540": [email_zeuxis],
    "0595": [email_zeuxis],
    "0529": [email_zeuxis],
    "0768": [email_luciano],
    "0749": [email_luciano],
    "0760": [email_luciano],
    "2559": [email_luciano],
    "0753": [email_luciano],
    "0662": [email_clarissa],
    "2585": [email_clarissa],
    "2511": [email_clarissa],
    "2522": [email_clarissa],
    "2574": [email_clarissa],
    "0571": [email_claudio],
    "0743": [email_claudio],
    "0713": [email_claudio],
    "2565": [email_claudio],
    "2572": [email_claudio],
    "0775": [email_ivan],
    "2557": [email_ivan],
    "2558": [email_ivan],
    "0647": [email_ivan],
    "0765": [email_ivan],
    "1071": [email_sandra, "henrique.silva@ammovarejo.com.br"],
    "10C5": [email_sandra, "henrique.silva@ammovarejo.com.br"],
    "1010": [email_sandra, "mateus.barboza@ammovarejo.com.br"],
    "10F3": [email_sandra, "michele.gouvea@ammovarejo.com.br"],
    "1012": [email_sandra, "mateus.barboza@ammovarejo.com.br"],
    "0706": [email_rodrigo],
    "0742": [email_rodrigo],
    "2512": [email_rodrigo],
    "2536": [email_rodrigo],
    "0641": [email_rodrigo],
}

In [14]:
# mask = df_kpis_loja['distributorId'] == '0742'

# df_store = df_kpis_loja[mask].fillna(0).copy()

# df_store

# Inicio do Relatorio

In [5]:
body = f"""Relatório em anexo. Versão em teste de validação. 

Favor conferir os valores e dar feedback para: {email_teste}

Observações:

- A referência de data é sempre o faturamento e não a criação do pedido.

- Atualmente o valor de Venda Loja (PDV) ainda não considera a venda cupom, mas passará a considerar no futuro.

- A Meta Loja (PDV) não contempla a meta do multicanal.
"""

NameError: name 'objeto_lido' is not defined

In [12]:
activate_email = False

for id_loja in regional_loja.keys():
    df_store_formated, df_vendedor_formated_month, df_vendedor_formated_day = kpi_data_formater.format_all_tables(
        id_loja=id_loja,
        df_kpis_loja=kpi_data_manager.df_kpis_loja,
        df_kpis_vendedor=kpi_data_manager.df_kpis_vendedor,
        df_nome_vendedor=kpi_data_manager.df_nome_vendedor,
    )

    if df_store_formated.size > 0:
        mask = kpi_data_manager.df_lojas["loja_id"] == id_loja
        email_loja: str = kpi_data_manager.df_lojas[mask]["email"].values[0]
        email_regional = regional_loja[id_loja]
        nome_loja: str = kpi_data_manager.df_lojas[mask]["nome_completo"].values[0]
        nome_loja = nome_loja.title()
        marca_loja: str = kpi_data_manager.df_lojas[mask]["marca"].values[0]
        marca_loja = marca_loja.title()
        print(id_loja, marca_loja, nome_loja)

        yesterday_date = datetime.datetime.today() - datetime.timedelta(days=1)
        yesterday_date_str = yesterday_date.strftime("%Y-%m-%d")

        path = "reports/" + yesterday_date_str.replace("/", "_") + "/"
        filename: str = "relatorio_" + nome_loja.replace(" ", "_") + "_" + yesterday_date_str.replace("/", "_")
        filename = (
            filename.replace("á", "a")
            .replace("á", "a")
            .replace("ã", "a")
            .replace("ó", "o")
            .replace("ô", "o")
            .replace("é", "e")
            .replace("ê", "e")
        )
        filename

        create_pdf_report(
            filename=filename,
            nome_loja=f"{marca_loja} - {nome_loja}",
            yesterday_date=yesterday_date,
            df_store_formated=df_store_formated,
            df_vendedor_formated_month=df_vendedor_formated_month,
            df_vendedor_formated_day=df_vendedor_formated_day,
        )

        if activate_email == True:
            for email in ["vinicius.voltolini@ammovarejo.com.br", email_loja] + email_regional:
                send_email(
                    filename=f"{filename}.pdf",
                    sender_email="inovacao@ammovarejo.com.br",
                    sender_password="@ed8Q124",
                    recipient_email=email,
                    subject=f"[TESTE] Report Diário Loja: {marca_loja} {nome_loja} - {yesterday_date.strftime('%d.%m.%Y')}",
                    body=body,
                )
    else:
        print(id_loja, "Sem vendas")
        if activate_email == True:
            for email in ["vinicius.voltolini@ammovarejo.com.br", email_loja] + email_regional:
                send_email(
                    sender_email="inovacao@ammovarejo.com.br",
                    sender_password="@ed8Q124",
                    recipient_email=email,
                    subject=f"[TESTE] Report Diário Loja: {marca_loja} {nome_loja} - {yesterday_date.strftime('%d.%m.%Y')}",
                    body="Sem vendas no dia anterior.",
                )

NameError: name 'regional_loja' is not defined

[ ] - 

In [1]:
from analytics_messenger.email_messenger.messenger import EmailReportSender


testeSender = EmailReportSender()
testeSender.send_report()

/Users/administrador/Library/Caches/pypoetry/virtualenvs/analytics-messenger-qF3K4D4l-py3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


{'document_file_name': 'relatorio_2560_-_Riomar_Recife_2024-09-04-report-content.pdf', 'document_content': [{'page': '1', 'content': [{'title': 'KPIs Loja 2560 - Riomar Recife - 04.09.2024', 'content':                                1          2             0
Período                      Dia        Mês           Ano
Venda Loja (PDV)        R$ 5.602  R$ 25.550  R$ 1.823.634
Meta Loja (PDV)        R$ 10.111  R$ 39.724  R$ 2.273.906
Meta % (PDV)                 55%        64%           80%
Markup                      3.07       3.04          2.91
Tickets/Dia                 12.0       13.2          16.6
Peças por Atendimento        2.3        2.5           3.0
Ticket Médio              R$ 467     R$ 482        R$ 442
Desconto Médio %              5%         9%           14%}]}, {'page': '2', 'content': [{'title': 'KPI Vendedor Mês', 'content':                                 1                 2              3   \
0                                                                     
Vende

In [6]:
class Teste:
    attr = 0

    def __init__(self) -> None:
        pass

    @classmethod
    def tiago(_):
        print(f"tiago {_.attr}")

    @staticmethod
    def convert_test_to_json():
        print(f"tiago2")

    @staticmethod
    def convert_json_to_test():
        print(f"tiago2")

    def tiago3(self):
        print(f"tiago3 {self.attr}")


abc = {"": ""}
jsonTeste = Teste.convert_test_to_json(abc)

tiago 0
tiago2
tiago3 0


In [8]:
# -> api/endpoint
# -> -> body { 'canal': 'email', type: 'venda_lojas_kpi' }
# -> -> -> return send_report(body)

# def send_report(body):
#   # deveria ter uma classe maior
#   canal = {
#     'email': EmailReportSender.send_report(body['type']),
#     # 'x': XReportSender().send_report(body['type']),
#     # 'y': YReportSender().send_report(body['type']),
#   }
#   canal[body['canal']]()

# class EncapsuladoraDeCanais:
#   def __init__(self) -> None:
#     pass

# def send_report(body):
#   encapsuladoraDeCanais = EncapsuladoraDeCanais()
#   encapsuladoraDeCanais.send_message(channel=body['canal'], report_type=body['type'])

In [ ]:
# class ReportDiarioEndpoint(IEndpoint):
#     @property
#     def endpoints(self) -> List[IEndpointConfig]:
#         return [IEndpointConfig(route="/reportdioaenpoitn", class_method=self.enviar, rest_method="POST", tags=["Get Info"]), body='valor']

#     def __init__(self) -> None:
#         pass

#     def enviar(self):
#         lista_de_DataFrames = kpi_data_manager.fetch_and_build_datasets_loja_kpi()

#         for loja in body['lista_de_lojas']:
#             self._enviar_para_uma_loja()

#     def _enviar_para_uma_loja():
#         resposta_formatada_em_obj:IReportContent = KpiDataFormater.format_kpi_data_to_pdf_file(lista_de_DataFrames)
#         endereco_do_pdf_pronto = PDFReportGenerator.generate_report(data=resposta_formatada_em_obj)

#         Messenger.send_email(
#             endereco_do_pdf=endereco_do_pdf_pronto,
#             email_loja='Coteminas',
#             nome_loja='Coteminas',
#             sbiject
#             body
#             atachcetmnt
#         )

## email regionais e lojas

In [1]:
email_luciano = "luciano.millamonte@coteminas.com.br"
email_clarissa = "clarissa.ferreira@ammovarejo.com.br"
email_claudio = "claudio.oliveira@ammovarejo.com.br"
email_ivan = "ivan.teixeira@ammovarejo.com.br"
email_zeuxis = "zeuxis.guimaraes@coteminas.com.br"
email_rodrigo = "rodrigo.graci@coteminas.com.br"
email_sandra = "sandra.pereira@ammovarejo.com.br"
email_teste = "joao.garcia@ammovarejo.com.br"

regional_loja = {
    "2560": [email_teste],
    "2510": [email_teste],
    # '2540': [email_teste],
    # '0595':[email_zeuxis],
    # '0529':[email_zeuxis],
    # '0768':[email_luciano],
    # '0749':[email_luciano],
    # '0760':[email_luciano],
    # '2559':[email_luciano],
    # '0753':[email_luciano],
    # '0662':[email_clarissa],
    # '2585':[email_clarissa],
    # '2511':[email_clarissa],
    # '2522':[email_clarissa],
    # '2574':[email_clarissa],
    # '0571':[email_claudio],
    # '0743':[email_claudio],
    # '0713':[email_claudio],
    # '2565':[email_claudio],
    # '2572':[email_claudio],
    # '0775':[email_ivan],
    # '2557':[email_ivan],
    # '2558':[email_ivan],
    # '0647':[email_ivan],
    # '0765':[email_ivan],
    # '1071':[email_sandra,'henrique.silva@ammovarejo.com.br'],
    # '10C5':[email_sandra,'henrique.silva@ammovarejo.com.br'],
    # '1010':[email_sandra,'mateus.barboza@ammovarejo.com.br'],
    # '10F3':[email_sandra,'michele.gouvea@ammovarejo.com.br'],
    # '1012':[email_sandra,'mateus.barboza@ammovarejo.com.br'],
    # '0706':[email_rodrigo],
    # '0742':[email_rodrigo],
    # '2512':[email_rodrigo],
    # '2536':[email_rodrigo],
    # '0641':[email_rodrigo]
}

In [4]:
from commom.data.data_manager import KpiDataManager
import pickle

dataManager = KpiDataManager()
dataManager.fetch_and_build_datasets(source="")

pickle.dump(dataManager, open("kpi_data_manager.pkl", "wb"))

/Users/administrador/Library/Caches/pypoetry/virtualenvs/innovation-api-kUUtbMIs-py3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [2]:
# dataManager.df_kpis_loja.head()

,distributorId,year,month,discount_value,net_value,store_cost,n_parcelas,tkts,amount,meta,tkts_per_day,type,mkp,tkt_medio,meta_percentual,PA,discount_percentual
0,0502,2024,9,468.54,15900.06,2997.11,NaN,9,33,18727.0,9.0,Dia,5.31,1766.673333,84.90,3.666667,2.95
1,0505,2024,9,1193.16,-22017.56,-5999.50,NaN,15,-38,10212.0,15.0,Dia,3.67,-1467.837333,-215.60,-2.533333,-5.42
2,0509,2024,9,396.01,3976.49,1696.40,NaN,13,25,4810.0,13.0,Dia,2.34,305.883846,82.67,1.923077,9.96
3,0514,2024,9,743.36,11916.34,3692.44,NaN,10,40,15823.0,10.0,Dia,3.23,1191.634000,75.31,4.0,6.24
4,0529,2024,9,502.74,2379.66,879.92,NaN,4,8,7423.0,4.0,Dia,2.70,594.915000,32.06,2.0,21.13


# API TESTE

## Post -> endpoint -> BigQuery + format + PDF -> send email

In [1]:
import pandas as pd
import datetime
from analytics_messenger.data.data_formater import KpiDataFormater
from analytics_messenger.email_messenger.report_generator import PDFReportGenerator
from analytics_messenger.email_messenger.messenger import EmailReportSender
import pickle


def enviar(dataframes, ids_loja: list[str]):

    for id in ids_loja:
        # obj_report_content = KpiDataFormater.format_kpi_data_to_report_content(
        #   id_loja= id,
        #   df_kpis_loja = objeto_lido.df_kpis_loja,
        #   df_kpis_vendedor = objeto_lido.df_kpis_vendedor,
        #   df_nome_vendedor = objeto_lido.df_nome_vendedor,
        #   df_lojas = objeto_lido.df_lojas,
        # )

        obj_report_content = ReportGenerator.generate_report_content(report="kpi_email")
        file_name = ReportGenerator.generate_pdf(report="kpi_email", obj_report_content=obj_report_content)
        email_body = ReportGenerator.generate_report_body(report="kpi_email", obj_report_content=obj_report_content)
        # body = ClasseXPTO.generate_email_body(email_body)

        for email in ["vinicius", email_regional, email_loja]:  # ->  gerenate_report_email_list(loja) -> List[str]
            EmailReportSender.send_report(
                subject="assunto do email",
                recipient="joao.garcia@ammovarejo.com.br",
                file_name=file_name,
                regional_email="teste@teste.com.br",
            )


with open("kpi_data_manager.pkl", "rb") as file:
    objeto_lido = pickle.load(file)

enviar(objeto_lido, "2510")

relatorio_2510_-_Riomar_Fortaleza_2024-09-08.pdf.pdf
-----------------------------------------------------------------------
-----------------------------------------------------------------------
-----------------------------------------------------------------------
assunto do email

      Relatório em anexo. Versão em teste de validação. 

      Favor conferir os valores e dar feedback para: teste@teste.com.br

      Observações:

      - A referência de data é sempre o faturamento e não a criação do pedido.

      - Atualmente o valor de Venda Loja (PDV) ainda não considera a venda cupom, mas passará a considerar no futuro.

      - A Meta Loja (PDV) não contempla a meta do multicanal.
    
joao.garcia@ammovarejo.com.br
relatorio_2510_-_Riomar_Fortaleza_2024-09-08.pdf.pdf


## Aqui ta certo

# API FINAL

- o codigo abaixo deve ser colocado no endpoint da api

In [1]:
from commom.report_generator.report_generator import ReportGenerator
from commom.data_classes.store_info_data_class import StoreInfo
from innovation_messenger.messenger import IEmailSender, Messenger


def enviar_email_kpi_diario(dataframes, ids_loja: list[str]):
    reportGenerator = ReportGenerator()

    yesterday_date = datetime.datetime.today() - datetime.timedelta(days=1)
    yesterday_date_str = yesterday_date.strftime("%d.%m.%Y").replace("/", "_")

    for id in ids_loja:
        store = StoreInfo(df_lojas=dataframes.df_lojas, id_loja=id)

        formated_content = reportGenerator.format_report_content(
            report_type="kpi",
            df_kpis_loja=dataframes.df_kpis_loja,
            df_kpis_vendedor=dataframes.df_kpis_vendedor,
            df_nome_vendedor=dataframes.df_nome_vendedor,
            yesterday_date=yesterday_date_str,
            store=store,
        )

        file_name = reportGenerator.generate_report(report_type="kpi", obj_report_content=formated_content)

        emails_recipients_list = reportGenerator.get_recipients_for_report(
            report_type="kpi",
            store=store,
            # test_email='joao.garcia@ammovarejo.com.br'
        )

        for email_recipient in emails_recipients_list:
            email_sender = IEmailSender(
                subject=reportGenerator.create_kpi_email_subject(store=store, report_date_string=yesterday_date_str),
                recipient=email_recipient,
                file_name=file_name,
                body=reportGenerator.create_kpi_email_body(store.email_regional),
            )

            # Messenger.send_report(
            #   report_channel='email',
            #   email_sender=email_sender,
            # )


local_dataframes = KpiDataManager().fetch_local_datasets("kpi_data_manager.pkl")
enviar_email_kpi_diario(local_dataframes, ["2560", "2510"])

ImportError: cannot import name 'XPos' from 'fpdf' (/Users/administrador/Library/Caches/pypoetry/virtualenvs/innovation-api-kUUtbMIs-py3.10/lib/python3.10/site-packages/fpdf/__init__.py)

In [1]:
import fpdf

In [13]:
from commom.data.data_manager import KpiDataManager

dataframes = KpiDataManager().fetch_local_datasets("kpi_data_manager.pkl")  # mock
last_update = dataframes.df_vendas["invoice_date"].max()

In [15]:
last_update

datetime.date(2024, 9, 10)

In [3]:
import datetime
# from commom.data.data_manager import KpiDataManager


kpi_data_manager = KpiDataManager()

if kpi_data_manager.should_fetch_datasets:
    kpi_data_manager.fetch_and_build_datasets()

NameError: name 'KpiDataManager' is not defined

In [1]:
from commom.data.data_manager import KpiDataManager

k = KpiDataManager()
# k._fetch_data_from_bigquery()
k.fetch_and_build_datasets()
# pickle.dump(k, open("kpi_data_manager2.pkl", "wb"))

/Users/administrador/Library/Caches/pypoetry/virtualenvs/innovation-api-kUUtbMIs-py3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


True

In [2]:
k._build_df_vendas_pdv()

In [3]:
k._build_df_parcelas_with_displaycode()

In [5]:
kpis_loja_dict = {}

for kpi_period in ["Dia", "Mês", "Ano"]:
    kpis_loja_dict[f"kpis_loja_{kpi_period.lower()}"] = k._build_kpis_dataframe(
        kpi_type="loja", kpi_period=kpi_period
    )
    

In [14]:
#  pickle.dump(k, open("kpi_data_manager2.pkl", "wb"))
mask_ativa = k.df_lojas['ativa']
k.df_lojas[mask_ativa]['loja_group_code'].unique()

array(['10C2', '1043', '1044', '1049', 'CDMM', 'CDAR', 'CDSA', '10C5',
       '0640', '1071', '10C4', '1012', '10C0', '1021', '10A8', '1010',
       '1099', '1028', '1055', '1017', '1018', '10F8', '1089', '1088',
       '1027', '1065', '10B8', '10F9', '10D0', '10E7', '10A1', '1054',
       '10A7', '1061', '10C3', '1082', '1093', '1060', '1045', '10G6',
       '1091', '10E8', '10G8', '1052', '0691', '10G9', '10H0', '10H3',
       '10H2', '10E1', '10D4', '2527', '1013', '10I2', '10I3', '0619',
       '10I7', '10L2', '10L4', '10L3', '10L5', '2535', '10I9', '0540',
       '1007', '0565', '0704', '10F3', '10H6', '0091', '10H7', '0632',
       '10J7', '10L0', '10L8', '10L7', '10H8', '0636', '2576', '0608',
       '10C1', '0652', '0628', '0774', '2529', '0599', '2550', '2574',
       '2519', '0524', '0554', '0743', '0772', '2522', '0629', '0589',
       '2585', '2541', '0593', '0626', '0709', '0557', '2532', '0655',
       '2531', '0583', '2548', '1020', '0525', '10L6', '5000', '5001',
      

In [15]:
# k.df_lojas.columns
k.df_lojas.loc[:,['ativa', 'status', 'loja_id']]

,ativa,status,loja_id
0,False,FECHADA,1023
1,False,FECHADA,0527
2,False,FECHADA,1067
3,False,FECHADA,0658
4,False,FECHADA,0651
...,...,...,...
449,True,ABERTA,2577
450,True,ABERTA,2583
451,False,FECHADA,2589
452,True,ABERTA,2590


In [17]:
import pickle
pickle.dump(k, open("kpi_data_manager2.pkl", "wb"))

## Fim API FINAL

- nada mais pra baixo deve ser adicionado na api

## outas coias

In [2]:
class StoreInfo:
    def __init__(self, df_lojas, id_loja):
        mask = df_lojas["loja_id"] == id_loja
        nome_loja: str = df_lojas[mask]["nome_completo"].values[0]
        self.nome_loja = nome_loja.title()
        marca_loja: str = df_lojas[mask]["marca"].values[0]
        self.marca_loja = marca_loja.title()
        tipo: str = df_lojas[mask]["tipo"].values[0]
        self.tipo = tipo.title()
        self.email: str = df_lojas[mask]["email"].values[0]
        regional: str = df_lojas[mask]["regional"].values[0]
        self.regional = regional.title()

In [22]:
info_loja = StoreInfo(df_lojas=objeto_lido.df_lojas, id_loja="1077")

In [23]:
info_loja.email

'reboucas@mmartan.com.br'

In [1]:
mask = objeto_lido.df_lojas["loja_id"] == "1077"
objeto_lido.df_lojas[mask].T

NameError: name 'objeto_lido' is not defined

In [3]:
from innovation_api.api.create_api import create_api


api = create_api()


In [7]:
from commom.config import config

config.credentials.service_account_email

'advanced-analytics-service@projetoomni.iam.gserviceaccount.com'